<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Soft-Voting-Edited-Dataset/blob/main/Cross_Project_SV_train_mina%2Btest_freeplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from numpy import mean
from numpy import std

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-mina data 2.0.7-2.0.8 - mina data 2.0.7-2.0.8.csv")


In [63]:
weights['change'] = weights['change'].map({'yes': 1, 'no': 0})

In [64]:
weights.shape

(610, 8)

In [65]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,53,15,0,100,4,0
1,10,0,391,74,2,66,12,0
2,25,2,194,225,0,98,181,0
3,4,0,14,47,0,66,1,0
4,34,0,53,1049,0,0,40,0


In [66]:
X = weights.drop('change', axis=1)
y = weights['change']

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [68]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [69]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [70]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.8415841584158416, 0.8415841584158416, 0.8415841584158416]


In [71]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-freeplane 1.7.9 - freeplane 1.7.9.csv")

In [72]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [73]:
data.shape

(1278, 8)

In [74]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,6,0,12,155,1,70,32,0
1,9,0,5,118,1,50,22,0
2,3,0,2,20,2,0,4,0
3,2,0,1,7,2,0,1,0
4,0,0,24,128,2,0,27,0


In [75]:
X = data.drop('change', axis=1)
y = data['change']

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [77]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [78]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 90.625


In [79]:
print(yhat)

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [80]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[229   0]
 [ 24   3]]


In [81]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = ensemble.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.8078602620087336


In [82]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.3171291955305963


In [83]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5555555555555556


In [84]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.3333333333333333


In [85]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       229
           1       1.00      0.11      0.20        27

    accuracy                           0.91       256
   macro avg       0.95      0.56      0.58       256
weighted avg       0.92      0.91      0.87       256

